In [1]:
from svm_game_win_predictor import *
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
year = 2022
nba_test = get_schedule(year, playoffs=False)
pbps = {}

game_summaries_test_teams = []
X_test_teams = []
y_test_teams = []

printouts=True

In [3]:
accuracies = []
predicted_team_records = {team: 0 for team in teams}

for _team in teams:
    team = _team.title() if _team != philly else 'Philadelphia 76ers'
    predicted_team_records[team] = predicted_team_records.pop(_team)

    #  Remove games that have not yet been played
    season = nba_test[~np.isnan(nba_test.VISITOR_PTS)]
    season = nba_test[~np.isnan(nba_test.HOME_PTS)]

    #  Select games played by a specific team if desired
    team_season = pd.concat((season[season.VISITOR == team], season[season.HOME== team]), axis=0)


    #  =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

    games_pbp = []
    wins = []
    game_summaries = []

    playcount_stopper = 200

    #  Extract parameters used by play-by-play functions 
    for idx,row in team_season.iterrows():
        date = row['DATE']
        away = name_to_abbr(row['VISITOR'])
        home = name_to_abbr(row['HOME'])
        home_winner = row['HOME_PTS'] > row['VISITOR_PTS']
        winner = home if home_winner else away

        summary = f'{date.date()}: {away} @ {home} | winner: {winner}'
        game_summaries.append(summary)

        wins.append(1 if winner == name_to_abbr(team) else 0)

        #  Begin processing play-by-play data on a per-game basis
        game_key = str(date) + str(home) + str(away)  # cache pbp data between uses to save time on the network

        if game_key not in pbps.keys():
            pbp = get_pbp(date, home, away)
            pbps[game_key] = pbp
        else:
            pbp = pbps[game_key]

        pbp_playcount = len(pbp)

        # Stop collecting pbp data after n plays 
        # Comment this line to use the full pbp data; 
        # this may introduce problems as teams play different number of games (reschedule, cancel, playoffs...)
        pbp_playcount = playcount_stopper


        plays = np.zeros((pbp_playcount, 6))  # [HOME?, time (secs), score, benefit, detriment, momentum_score]    

        for play in range(pbp_playcount):
            elapsed_gametime = calculate_elapsed_time(pbp.iloc[play][TIME_IDX], pbp.iloc[play][QTR_IDX])
            action_home = pbp.iloc[play][HOME_IDX]
            action_away = pbp.iloc[play][AWAY_IDX]

            if action_home is not np.nan:
                plays[play] = np.hstack((1, process_play(elapsed_gametime.seconds, action_home.lower()), 0))
            else:
                plays[play] = np.hstack((0, process_play(elapsed_gametime.seconds, action_away.lower()), 0))

        plays = momentum_checks(plays)
        plays = plays / plays.max(axis=0)

        games_pbp.append(plays)

    #  =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

#     if year == 2021:  # In 2020-21, the regular seasons was capped at 72 games instead of 82.
#         games_pbp = games_pbp[:25]
#         wins = wins[:25]
#         game_summaries = game_summaries_test_teams[:25]

    if printouts:
        print(team)
#         print(np.array(games_pbp).shape)     

    game_count = len(games_pbp)

    games_pbp, wins, game_summaries = shuffle_arrays(np.array(games_pbp), np.array(wins), game_summaries)
    
    game_summaries_test_teams.append(game_summaries)

    X_test = np.array(games_pbp)[:].reshape(game_count,-1)
    y_test = np.array(wins)[:].reshape(-1,)
    
    X_test_teams.append(X_test)
    y_test_teams.append(y_test)

    
len(X_test_teams)

Philadelphia 76ers
Atlanta Hawks
Boston Celtics
Brooklyn Nets
Chicago Bulls
Charlotte Hornets
Cleveland Cavaliers
Dallas Mavericks
Denver Nuggets
Detroit Pistons
Golden State Warriors
Houston Rockets
Indiana Pacers
Los Angeles Clippers
Los Angeles Lakers
Memphis Grizzlies
Miami Heat
Milwaukee Bucks
Minnesota Timberwolves
New Orleans Pelicans
New York Knicks
Oklahoma City Thunder
Orlando Magic
Phoenix Suns
Portland Trail Blazers
Sacramento Kings
San Antonio Spurs
Toronto Raptors
Utah Jazz
Washington Wizards


30

In [32]:
def run1(model, printouts=True):
    year=2021
    nba = get_schedule(year, playoffs=False)
    
    accuracies = []
    predicted_team_records = {team: 0 for team in teams}

    team_idx = 0
    
    for _team in teams:
        team = _team.title() if _team != philly else 'Philadelphia 76ers'
        predicted_team_records[team] = predicted_team_records.pop(_team)
        
        #  Remove games that have not yet been played
        season = nba[~np.isnan(nba.VISITOR_PTS)]
        season = nba[~np.isnan(nba.HOME_PTS)]

        #  Select games played by a specific team if desired
        team_season = pd.concat((season[season.VISITOR == team], season[season.HOME== team]), axis=0)


        #  =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

        games_pbp = []
        wins = []
        game_summaries = []

        playcount_stopper = 200

        #  Extract parameters used by play-by-play functions 
        for idx,row in team_season.iterrows():
            date = row['DATE']
            away = name_to_abbr(row['VISITOR'])
            home = name_to_abbr(row['HOME'])
            home_winner = row['HOME_PTS'] > row['VISITOR_PTS']
            winner = home if home_winner else away

            summary = f'{date.date()}: {away} @ {home} | winner: {winner}'
            game_summaries.append(summary)

            wins.append(1 if winner == name_to_abbr(team) else 0)

            #  Begin processing play-by-play data on a per-game basis
            game_key = str(date) + str(home) + str(away)  # cache pbp data between uses to save time on the network
            
            if game_key not in pbps.keys():
                pbp = get_pbp(date, home, away)
                pbps[game_key] = pbp
            else:
                pbp = pbps[game_key]
                
            pbp_playcount = len(pbp)

            # Stop collecting pbp data after n plays 
            # Comment this line to use the full pbp data; 
            # this may introduce problems as teams play different number of games (reschedule, cancel, playoffs...)
            pbp_playcount = playcount_stopper


            plays = np.zeros((pbp_playcount, 6))  # [HOME?, time (secs), score, benefit, detriment, momentum_score]    

            for play in range(pbp_playcount):
                elapsed_gametime = calculate_elapsed_time(pbp.iloc[play][TIME_IDX], pbp.iloc[play][QTR_IDX])
                action_home = pbp.iloc[play][HOME_IDX]
                action_away = pbp.iloc[play][AWAY_IDX]

                if action_home is not np.nan:
                    plays[play] = np.hstack((1, process_play(elapsed_gametime.seconds, action_home.lower()), 0))
                else:
                    plays[play] = np.hstack((0, process_play(elapsed_gametime.seconds, action_away.lower()), 0))

            plays = momentum_checks(plays)
            plays = plays / plays.max(axis=0)

            games_pbp.append(plays)

        #  =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

        if year == 2021:  # In 2020-21, the regular seasons was capped at 72 games instead of 82.
            games_pbp = games_pbp[:72]
            wins = wins[:72]
            game_summaries = game_summaries[:72]
        
        if printouts:
                print(np.array(games_pbp).shape)     
        
        game_count = len(games_pbp)

        games_pbp, wins, game_summaries = shuffle_arrays(np.array(games_pbp), np.array(wins), game_summaries)

        tts_idx = round(game_count*0.75)  # train-test-split index

        X_train = np.array(games_pbp)[:tts_idx].reshape(tts_idx,-1)
        y_train = np.array(wins)[:tts_idx].reshape(-1,)


        #  =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

        clf = model.fit(X_train, y_train)
        if not (team_idx+1 % 10):
            print(team_idx)
        preds = clf.predict(X_test_teams[team_idx])
        acc = accuracy_score(y_test_teams[team_idx], preds)
        accuracies.append(acc)
        
        test_game_count = 25

        if printouts:
                print(f'Team: {team} accuracy = {100*acc}%')

        test_predictions = [results[i] for i in preds]

        for i in range(test_game_count):
            if preds[i] == 1:
                predicted_team_records[team] += 1
            if printouts:
#                 print(f'{game_summaries[i]} ||| Prediction: {name_to_abbr(team)} {test_predictions[i]}')
                print(f'{game_summaries_test_teams[team_idx][i]} ||| Prediction: {name_to_abbr(team)} {test_predictions[i]}')


        #  =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
        team_idx+=1  # end of outer for loop, increment idx (used for indexing the test data)
        #  =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
    
    
    test_games = test_game_count  # The number of games that were used for testing
    
    league_accuracy = np.mean(accuracies)*100
    league_wins_avg = np.mean(list(predicted_team_records.values()))/test_game_count
    
    if printouts:
        print(f'test_games = {test_games}')
        print(f'League accuracy average: {league_accuracy:.2f}%')
        print(f'League wins average: {league_wins_avg:.3f}%')


        for team in range(len(teams)):
            _team = teams[team].title() if teams[team] != philly else 'Philadelphia 76ers'
            print(f'({predicted_team_records[_team]}) Win rate: {(predicted_team_records[_team]/test_games):.3f} | Correctly predicted {accuracies[team]*100:.2f}% for team {teams[team]}')

    return league_accuracy, league_wins_avg, accuracies, predicted_team_records

In [55]:
%%time

pows = [-2, -3]

ranges = []
for power in pows:
    ranges += list(np.arange(pow(10,power), pow(10,power+1), pow(10,power)*5))

ranges = [n/10 for n in ranges]
gammas = ranges
cs = ranges
    
for c in cs:
    for gamma in gammas:

        model = SVC(random_state=0, kernel='poly', degree=4, C=C, gamma=gamma)
        league_accuracy, league_wins_avg, accuracies, predicted_team_records = run1(model, printouts=False)

        print(f'c = {c}, gamma = {gamma}')
        print(f'League accuracy average: {np.mean(accuracies)*100:.2f}%')
        print(f'League wins average: {np.mean(list(predicted_team_records.values()))/25:.3f}%')
        print(f'League win rate standard deviation: {np.std(list(predicted_team_records.values()))}')
        print('------------------------------------------------------------')

c = 0.001, gamma = 0.001
League accuracy average: 58.39%
League wins average: 0.533%
League win rate standard deviation: 12.163698268025048
------------------------------------------------------------
c = 0.001, gamma = 0.006
League accuracy average: 58.05%
League wins average: 0.469%
League win rate standard deviation: 11.938546347394599
------------------------------------------------------------
c = 0.001, gamma = 0.0001
League accuracy average: 57.32%
League wins average: 0.505%
League win rate standard deviation: 12.073340695745962
------------------------------------------------------------
c = 0.001, gamma = 0.0006000000000000001
League accuracy average: 58.61%
League wins average: 0.480%
League win rate standard deviation: 12.231107881136523
------------------------------------------------------------
c = 0.006, gamma = 0.001
League accuracy average: 58.84%
League wins average: 0.485%
League win rate standard deviation: 12.197631828442036
---------------------------------------

In [52]:
%%time

pows = [0, -1, -2, -3]

ranges = []
for power in pows:
    
    C = 0.005
    gamma = pow(10,power)

    model = SVC(random_state=0, kernel='poly', degree=4, C=C, gamma=gamma)
    league_accuracy, league_wins_avg, accuracies, predicted_team_records = run1(model, printouts=False)


    print(f'League accuracy average: {np.mean(accuracies)*100:.2f}%')
    print(f'League wins average: {np.mean(list(predicted_team_records.values()))/25:.3f}%')
    print(f'League win rate standard deviation: {np.std(list(predicted_team_records.values()))}')
    print('------------------------------------------------------------')

League accuracy average: 58.80%
League wins average: 0.464%
League win rate standard deviation: 10.252154245165583
------------------------------------------------------------
League accuracy average: 55.75%
League wins average: 0.437%
League win rate standard deviation: 9.65033102483479
------------------------------------------------------------
League accuracy average: 57.14%
League wins average: 0.453%
League win rate standard deviation: 11.264496832477201
------------------------------------------------------------
League accuracy average: 58.42%
League wins average: 0.499%
League win rate standard deviation: 12.211288038350236
------------------------------------------------------------
CPU times: user 11min 2s, sys: 2.76 s, total: 11min 5s
Wall time: 11min 9s
